<a href="https://colab.research.google.com/github/nephelim74/PythonDS/blob/main/DZ8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

1. Предобработка данных:  
— Очистите данные от пропусков и аномалий.  
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

In [83]:
df = pd.read_csv('/content/Sleep_health_and_lifestyle_dataset.csv')

In [84]:
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


1. Person ID: Уникальный идентификатор для каждого участника исследования. Это позволяет отслеживать данные конкретного человека.

2. Gender: Пол участника (например, мужской или женский). Это поле может использоваться для анализа различий в здоровье и образе жизни между мужчинами и женщинами.

3. Age: Возраст участника. Это важный фактор, влияющий на здоровье, качество сна и уровень физической активности.

4. Occupation: Профессия участника. Разные профессии могут иметь разные уровни стресса и физической активности, что может влиять на здоровье и качество сна.

5. Sleep Duration: Продолжительность сна, измеряемая в часах. Это важный показатель, который может влиять на общее состояние здоровья и качество жизни.

6. Quality of Sleep: Качество сна, которое может оцениваться по различным критериям, таким как частота пробуждений, глубина сна и субъективное восприятие. Это поле помогает понять, насколько восстановительным был сон.

7. Physical Activity Level: Уровень физической активности участника, который может варьироваться от малоподвижного образа жизни до высокой физической активности. Это может влиять на общее здоровье и качество сна.

8. Stress Level: Уровень стресса, который может быть оценен по шкале или другими методами. Высокий уровень стресса может негативно сказываться на качестве сна и общем состоянии здоровья.

9. BMI Category: Категория индекса массы тела (ИМТ), которая может быть нормальной, избыточной, недостаточной или ожирением. ИМТ является важным показателем здоровья.

10. Blood Pressure: Артериальное давление, которое является важным показателем сердечно-сосудистого здоровья. Может быть представлено как систолическое и диастолическое давление.

11. Heart Rate: Частота сердечных сокращений, которая может указывать на уровень физической активности и общее состояние сердечно-сосудистой системы.

12. Daily Steps: Количество шагов, сделанных участником за день. Это показатель физической активности и может быть связан с общим состоянием здоровья.

13. Sleep Disorder: Наличие или отсутствие расстройства сна, такого как бессонница или апноэ (ОАС). Это поле может помочь в анализе факторов, влияющих на качество сна.

In [86]:
df.duplicated().sum()

0

In [87]:
df['Sleep Disorder'].value_counts(dropna=False)

,count
Sleep Disorder,
NaN,219
Sleep Apnea,78
Insomnia,77


Пропусков в датасете фактически нет, но в колонке Sleep Disorder следует заменить NaN на No, тогда она будет отражать суть. К тому же это наша целевая колонка и мы не можем разбивать ее на три колонки.
Кодирования также требуют колонки Gender, Occupation, BMI Category.  
Также потребуется что-то сделать с колонкой Blood Pressure, так как пока она не понятна для модели.

In [88]:
df['Sleep Disorder'] = df['Sleep Disorder']. fillna ('No')

In [89]:
df['Sleep Disorder'].value_counts(dropna=False)

,count
Sleep Disorder,
No,219
Sleep Apnea,78
Insomnia,77


In [90]:
# from sklearn. preprocessing import OneHotEncoder

# encoder = OneHotEncoder(handle_unknown='ignore')
# encoder_df = pd.DataFrame(encoder. fit_transform(df[['Sleep Disorder']]). toarray ())
# df = pd.concat([df, encoder_df], axis=1).drop('Sleep Disorder', axis=1)
# df.rename(columns={0: 'Insomnia', 1: 'No_Disorder', 2: 'Sleep Apnea'}, inplace = True )

In [91]:
df['Sleep_Disorder'] = df['Sleep Disorder'].map({
    'No': 0,
    'Sleep Apnea': 2,
    'Insomnia': 1
})
df = df.drop('Sleep Disorder', axis=1)

In [92]:
df['Sleep_Disorder'].value_counts(dropna=False)

,count
Sleep_Disorder,
0,219
2,78
1,77


In [93]:
df['Gender'].value_counts(dropna=False)

,count
Gender,
Male,189
Female,185


In [94]:
df['BMI Category'].value_counts(dropna=False)

,count
BMI Category,
Normal,195
Overweight,148
Normal Weight,21
Obese,10


In [95]:
df.loc[df['BMI Category'] == 'Normal Weight', 'BMI Category'] = 'Normal'

In [96]:
from sklearn.preprocessing import LabelEncoder
selected_columns = df.loc[:, ['Gender', 'BMI Category']]
label_encoder = LabelEncoder()

# Кодирование столбцов
encoded_columns = {}
for column in selected_columns:
    df[column + '_encoded'] = label_encoder.fit_transform(df[column])
    # Сохраняем маппинг
    encoded_columns[column] = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Вывод DataFrame
display(df)

# Вывод легенды кодировки для каждого столбца
for column, mapping in encoded_columns.items():
    print(f"Кодировка для {column}:", mapping)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,BMI Category_encoded
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,0,1,2
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0,1,0
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,0,1,0
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,2,1,1
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,2,0,2
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,2,0,2


Кодировка для Gender: {'Female': 0, 'Male': 1}
Кодировка для BMI Category: {'Normal': 0, 'Obese': 1, 'Overweight': 2}


In [97]:
df.drop(['Gender', 'BMI Category'], axis = 1, inplace=True)

In [98]:
df['Occupation'].value_counts(dropna=False)

,count
Occupation,
Nurse,73
Doctor,71
Engineer,63
Lawyer,47
Teacher,40
Accountant,37
Salesperson,32
Software Engineer,4
Scientist,4


In [99]:
df.loc[df['Occupation'] == 'Sales Representative', 'Occupation'] = 'Salesperson'

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание экземпляра TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование документов в TF-IDF векторы
tfidf_matrix = vectorizer.fit_transform(df['Occupation'])
# Просмотр результатов
print(tfidf_matrix)

# Для получения словаря терминов
print(vectorizer.get_feature_names_out())

  (0, 8)	0.8911405202917568
  (0, 2)	0.45372742158055335
  (1, 1)	1.0
  (2, 1)	1.0
  (3, 6)	1.0
  (4, 6)	1.0
  (5, 8)	0.8911405202917568
  (5, 2)	0.45372742158055335
  (6, 9)	1.0
  (7, 1)	1.0
  (8, 1)	1.0
  (9, 1)	1.0
  (10, 1)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 1)	1.0
  (15, 1)	1.0
  (16, 5)	1.0
  (17, 1)	1.0
  (18, 5)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  (21, 1)	1.0
  (22, 1)	1.0
  :	:
  (349, 5)	1.0
  (350, 5)	1.0
  (351, 5)	1.0
  (352, 5)	1.0
  (353, 5)	1.0
  (354, 5)	1.0
  (355, 5)	1.0
  (356, 5)	1.0
  (357, 5)	1.0
  (358, 5)	1.0
  (359, 5)	1.0
  (360, 5)	1.0
  (361, 5)	1.0
  (362, 5)	1.0
  (363, 5)	1.0
  (364, 5)	1.0
  (365, 5)	1.0
  (366, 5)	1.0
  (367, 5)	1.0
  (368, 5)	1.0
  (369, 5)	1.0
  (370, 5)	1.0
  (371, 5)	1.0
  (372, 5)	1.0
  (373, 5)	1.0
['accountant' 'doctor' 'engineer' 'lawyer' 'manager' 'nurse' 'salesperson'
 'scientist' 'software' 'teacher']


In [101]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Конкатенируем исходный DataFrame с новым DataFrame векторов TF-IDF
df = pd.concat([df, tfidf_df], axis=1).drop('Occupation', axis=1)

In [102]:
df

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Blood Pressure,Heart Rate,Daily Steps,Sleep_Disorder,...,accountant,doctor,engineer,lawyer,manager,nurse,salesperson,scientist,software,teacher
0,1,27,6.1,6,42,6,126/83,77,4200,0,...,0.0,0.0,0.453727,0.0,0.0,0.0,0.0,0.0,0.891141,0.0
1,2,28,6.2,6,60,8,125/80,75,10000,0,...,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,3,28,6.2,6,60,8,125/80,75,10000,0,...,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,4,28,5.9,4,30,8,140/90,85,3000,2,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
4,5,28,5.9,4,30,8,140/90,85,3000,2,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
370,371,59,8.0,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
371,372,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0
372,373,59,8.1,9,75,3,140/95,68,7000,2,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0


2. Генерация новых признаков:  
— Создайте комбинированные признаки, которые могут помочь улучшить модель.  
— Используйте полиномиальные признаки для числовых переменных.

In [103]:
df['systolic_pressure'] = df['Blood Pressure'].str.split('/').str[0].astype(int)
df['diastolic_pressure'] = df['Blood Pressure'].str.split('/').str[1].astype(int)

In [104]:
df = df.drop('Blood Pressure', axis=1)

In [105]:
df

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,...,engineer,lawyer,manager,nurse,salesperson,scientist,software,teacher,systolic_pressure,diastolic_pressure
0,1,27,6.1,6,42,6,77,4200,0,1,...,0.453727,0.0,0.0,0.0,0.0,0.0,0.891141,0.0,126,83
1,2,28,6.2,6,60,8,75,10000,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,125,80
2,3,28,6.2,6,60,8,75,10000,0,1,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,125,80
3,4,28,5.9,4,30,8,85,3000,2,1,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,140,90
4,5,28,5.9,4,30,8,85,3000,2,1,...,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
370,371,59,8.0,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
371,372,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95
372,373,59,8.1,9,75,3,68,7000,2,0,...,0.000000,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,140,95


Я хочу умножить продолжительность сна на его качество, чтобы получить эффективность сна.

In [106]:
df['Sleep_eff'] = df['Sleep Duration'] * df['Quality of Sleep']

Создадим колонку Lifestyle, которая будет комбинировать уровень физической активности и количество шагов в день.

In [107]:
# Нормализация данных
scaler = MinMaxScaler()
df_scale = df[['Physical Activity Level', 'Daily Steps']]
df_scale[['Physical Activity Level', 'Daily Steps']] = scaler.fit_transform(df[['Physical Activity Level', 'Daily Steps']])
# • scaler.fit_transform(...): Метод fit_transform выполняет две операции:
# • fit: Вычисляет минимальные и максимальные значения для каждого из столбцов, которые будут использоваться для масштабирования.
# • transform: Применяет нормализацию к данным, используя вычисленные минимальные и максимальные значения.
# • Обновление DataFrame: Результат масштабирования (нормализованные значения) присваивается обратно в столбцы Physical Activity Level и Daily Steps в df_scale.

# Выставление весов
activity_weight = 0.7  # Больший вес, так как зарплата важнее
steps_weight = 0.3  # Меньший вес, так как можно заниматься в зале, плавать и тп. и при этом немного ходить

# Вычисление взвешенной суммы
df['lifestyle'] = df_scale['Physical Activity Level'] * activity_weight + df_scale['Daily Steps'] * steps_weight

df.head()

#  результате выполнения этого кода:

# 1. Устанавливаются веса для уровня физической активности и количества шагов.

# 2. Вычисляется взвешенная сумма для каждого наблюдения, которая отражает общий уровень "образа жизни" на основе заданных весов.

# 3. Новый столбец 'lifestyle' добавляется в DataFrame df, и его значения будут отражать комбинацию уровня физической активности и количества шагов с учетом заданных весов.

# Это может быть полезно для дальнейшего анализа или моделирования, где вам нужно учитывать влияние различных факторов на общий уровень активности или образа жизни.


,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep_Disorder,Gender_encoded,...,manager,nurse,salesperson,scientist,software,teacher,systolic_pressure,diastolic_pressure,Sleep_eff,lifestyle
0,1,27,6.1,6,42,6,77,4200,0,1,...,0.0,0.0,0.0,0.0,0.891141,0.0,126,83,36.6,0.191429
1,2,28,6.2,6,60,8,75,10000,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,125,80,37.2,0.650000
2,3,28,6.2,6,60,8,75,10000,0,1,...,0.0,0.0,0.0,0.0,0.000000,0.0,125,80,37.2,0.650000
3,4,28,5.9,4,30,8,85,3000,2,1,...,0.0,0.0,1.0,0.0,0.000000,0.0,140,90,23.6,0.000000
4,5,28,5.9,4,30,8,85,3000,2,1,...,0.0,0.0,1.0,0.0,0.000000,0.0,140,90,23.6,0.000000


3. Отбор признаков:  
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).  
— Сравните качество модели до и после отбора признаков.

In [108]:
X = df.drop('Sleep_Disorder', axis=1)
y = df['Sleep_Disorder']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [109]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
# • DecisionTreeRegressor: Это класс из библиотеки sklearn, который используется для создания модели регрессии на основе дерева решений. Дерево решений — это метод, который разбивает данные на подмножества,
# основываясь на значениях признаков, и используется для предсказания числовых значений.

tree = DecisionTreeRegressor()
# Здесь создается объект tree, который представляет модель дерева решений для регрессии. На этом этапе модель еще не обучена

tree.fit(X_train, y_train)
# • Метод fit обучает модель на тренировочных данных:
#   • X_train: Это матрица признаков (независимые переменные) для обучающего набора данных.
#   • y_train: Это вектор целевых значений (зависимая переменная) для обучающего набора данных.
# • После выполнения этого метода модель будет готова к предсказаниям на новых данных.

prediction = tree.predict(X_test)
# • Метод predict используется для получения предсказаний на тестовых данных:
#   • X_test: Это матрица признаков (независимые переменные) для тестового набора данных.
# • Результат предсказания сохраняется в переменной prediction.

print(f'R^2 на тренировочных данных {r2_score(y_train, tree.predict(X_train))}')
print(f'R^2 на тестовых данных {r2_score(y_test, tree.predict(X_test))}')
# • Здесь вычисляется и выводится коэффициент детерминации  R²  для тренировочных и тестовых данных:
# • r2_score(y_train, tree.predict(X_train)): Вычисляет  R²  для тренировочных данных, используя фактические значения y_train и предсказания, полученные от модели на X_train.
# • r2_score(y_test, tree.predict(X_test)): Вычисляет  R²  для тестовых данных, используя фактические значения y_test и предсказания, полученные от модели на X_test.

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


▎Результаты

•  R²  на тренировочных данных: 1.0

•  R²  на тестовых данных: 0.8209 (примерно)

▎Интерпретация

1. ( R^2 = 1.0 на тренировочных данных:

   • Значение  R²  равное 1.0 означает, что модель идеально объясняет вариацию целевой переменной в тренировочных данных. Все предсказанные значения совпадают с фактическими значениями, и модель не имеет ошибок.

   • Это может указывать на то, что модель очень хорошо подстроилась под тренировочные данные, но также может сигнализировать о переобучении (overfitting), особенно если модель слишком сложна (например, если используется слишком глубокое дерево решений). Переобученная модель будет хорошо работать на тренировочных данных, но может не обобщаться на новые, невидимые данные.

2. ( R^2 \approx 0.8209 на тестовых данных:

   • Значение  R²  около 0.82 на тестовых данных означает, что модель объясняет примерно 82.09% вариации целевой переменной в тестовом наборе данных. Это довольно хороший результат, указывающий на то, что модель справляется с предсказанием значений на новых данных.

   • Однако это значение также ниже, чем 1.0, что говорит о том, что модель не идеальна и есть определенные ошибки в предсказаниях. Это может быть нормальным явлением, особенно если модель была переобучена на тренировочных данных.

▎Заключение

• Переобучение: Высокое значение  R²  на тренировочных данных (1.0) в сочетании с более низким значением на тестовых данных (0.82) может указывать на то, что модель переобучена. Это значит, что она хорошо подстраивается под конкретные данные, но может не так хорошо работать с новыми данными.

• Хорошая производительность: Тем не менее, значение  R²  на тестовых данных (0.82) указывает на то, что модель все же имеет хорошую предсказательную способность и может быть полезной для практического применения.

С настройками по умолчанию модель кажется переобученной, поскольку мы получаем 1 на тренировочных данных. При этом на тестовых данных коэффициент детерминации остается достаточно высоким.

In [110]:
from sklearn.feature_selection import RFE
# Создаем RFE модель
rfe = RFE(estimator=tree, n_features_to_select=13)
# estimator=tree: Здесь вы передаете ранее созданную модель дерева решений (tree) в качестве оценщика,
# который будет использоваться для оценки важности признаков.
# RFE будет использовать эту модель для определения, какие признаки являются наиболее важными.

# Обучение модели RFE
rfe = rfe.fit(X, y)
# • Метод fit обучает модель RFE на данных:
# • X: Матрица признаков (независимые переменные).
# • y: Вектор целевой переменной (зависимая переменная).
# • В процессе обучения RFE будет определять, какие признаки являются наиболее значимыми, и исключать наименее важные.

# Показываем какие признаки выбраны
selected_features = pd.DataFrame({
    'Feature': X.columns,  # Здесь X.columns должен содержать названия столбцов, использованных для X
    'Ranking': rfe.ranking_
})
#   • Здесь создается новый DataFrame selected_features, который содержит два столбца:
#   • Feature: Названия признаков из исходного DataFrame X.
#   • Ranking: Ранг каждого признака, присвоенный RFE. Признаки, которые были выбраны, будут иметь ранг 1,
#   а те, которые были исключены, будут иметь более высокие значения ранга.
print(selected_features.sort_values(by='Ranking', ascending=False))

                    Feature  Ranking
23                lifestyle       12
11                   doctor       11
10               accountant       10
18                 software        9
8            Gender_encoded        8
17                scientist        7
3          Quality of Sleep        6
5              Stress Level        5
4   Physical Activity Level        4
14                  manager        3
13                   lawyer        2
16              salesperson        1
22                Sleep_eff        1
21       diastolic_pressure        1
20        systolic_pressure        1
19                  teacher        1
0                 Person ID        1
15                    nurse        1
1                       Age        1
9      BMI Category_encoded        1
7               Daily Steps        1
6                Heart Rate        1
2            Sleep Duration        1
12                 engineer        1


In [111]:
top_features = selected_features.sort_values(by='Ranking', ascending=False)[:12]
top_features.head(100)

,Feature,Ranking
23,lifestyle,12
11,doctor,11
10,accountant,10
18,software,9
8,Gender_encoded,8
17,scientist,7
3,Quality of Sleep,6
5,Stress Level,5
4,Physical Activity Level,4
14,manager,3


In [112]:
X_train[top_features.Feature].head()

,lifestyle,doctor,accountant,software,Gender_encoded,scientist,Quality of Sleep,Stress Level,Physical Activity Level,manager,lawyer,salesperson
19,0.739286,1.0,0.0,0.0,1,0.0,7,6,75,0.0,0.0,0.0
357,0.696429,0.0,0.0,0.0,0,0.0,9,3,75,0.0,0.0,0.0
79,0.085714,1.0,0.0,0.0,1,0.0,6,8,30,0.0,0.0,0.0
167,0.420238,0.0,0.0,0.0,1,0.0,7,6,55,0.0,1.0,0.0
18,0.159524,0.0,0.0,0.0,0,0.0,5,7,40,0.0,0.0,0.0


In [113]:
top_features = selected_features.sort_values(by='Ranking', ascending=False)[:12]
# • Здесь selected_features — это DataFrame, который содержит названия признаков и их ранжирование, полученное из предыдущего шага с использованием метода рекурсивного исключения признаков (RFE).
# • sort_values(by='Ranking', ascending=False) сортирует признаки по их рангу в порядке убывания, что означает, что наиболее важные признаки (с рангом > 1) будут вверху.
# • [:12] выбирает верхние 12 признаков, которые имеют наименьший ранг (то есть наиболее значимые).

X_train_selected = X_train[top_features.Feature]
X_test_selected = X_test[top_features.Feature]
# • Здесь создаются новые матрицы признаков для тренировочных и тестовых данных, содержащие только отобранные 12 признаков.
# • X_train_selected: Это подмножество признаков из тренировочного набора данных.
# • X_test_selected: Это подмножество признаков из тестового набора данных.

# Обучение дерева решений на отобранных признаках
dt1 = DecisionTreeRegressor(random_state=42)
dt1.fit(X_train_selected, y_train)
#• Создается объект dt1, который представляет собой модель дерева решений для регрессии.
#• Метод fit обучает модель на отобранных признаках X_train_selected и целевых значениях y_train.

print(f'R^2 на тренировочных данных {r2_score(y_train, dt1.predict(X_train_selected))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt1.predict(X_test_selected))}')

R^2 на тренировочных данных 0.7828128560147208
R^2 на тестовых данных 0.6738575889818835


Результаты

•  R²  на тренировочных данных: 0.8142 (приблизительно)

•  R²  на тестовых данных: 0.7095 (приблизительно)

▎Интерпретация

1. ( R^2 = 0.8142 на тренировочных данных:

   • Значение  R²  равное 0.8142 означает, что модель объясняет примерно 81.42% вариации целевой переменной в тренировочных данных. Это довольно хороший результат, указывающий на то, что модель достаточно эффективно подстраивается под данные, на которых она была обучена.

   • Значение близкое к 1 указывает на то, что модель хорошо подходит к данным, и предсказания имеют высокую точность в пределах тренировочного набора.

2. ( R^2 = 0.7095 на тестовых данных:

   • Значение  R²  равное 0.7095 означает, что модель объясняет примерно 70.95% вариации целевой переменной в тестовом наборе данных. Это также хороший результат, но он ниже, чем на тренировочных данных.

   • Это может указывать на то, что модель не так хорошо обобщает на новые данные по сравнению с тем, как она работает на данных, на которых она была обучена.

▎Сравнение значений R²

• Разница между тренировочными и тестовыми данными:

  • Разница между значениями  R²  для тренировочных (0.8142) и тестовых (0.7095) данных может указывать на небольшое переобучение (overfitting). Это означает, что модель хорошо подстраивается под тренировочные данные, но может не так хорошо работать на новых, невидимых данных.

  • Хотя разница не является критической, она все же указывает на то, что модель могла бы быть улучшена для лучшего обобщения.

▎Заключение

• Хорошая производительность: В целом, значения  R²  в диапазоне от 0.7 до 0.8 считаются хорошими для многих задач регрессии, особенно если данные имеют сложные зависимости.

• Потенциал для улучшения: Рассмотрите возможность использования методов регуляризации, изменения архитектуры модели или добавления новых признаков, чтобы улучшить обобщающую способность модели и уменьшить разницу между тренировочными и тестовыми данными.

Таким образом, ваша модель показывает хорошую производительность, но есть возможность для улучшения, особенно в плане обобщения на новые данные.

Если обучать модель на топ 12 признаках (половине), то мы как будто бы избавляемся от переобучения. К тому же, разница между коэффициентом детерминации на тестовых и тренировочных данных сокращается.

In [114]:
feature_importances = pd.Series(tree.feature_importances_, index=X_train.columns)
# tree.feature_importances_: Это атрибут модели дерева решений, который содержит значения важности для каждого признака, используемого в модели. Важность признаков указывает на то, насколько каждый признак способствовал улучшению предсказаний модели.
# Чем выше значение важности, тем больше вклад данного признака в модель.

# Отбор 12 наиболее важных признаков
top_features2 = feature_importances.nlargest(12).index
top_features2
# • feature_importances.nlargest(12): Этот метод возвращает 12 наибольших значений важности признаков из feature_importances, а также их соответствующие индексы (названия признаков).
# Это позволяет выбрать 12 наиболее значимых признаков, которые имеют наибольшую важность для модели.
# • .index: Этот атрибут возвращает индексы (названия признаков) для 12 наиболее важных признаков. В результате переменная top_features2 будет содержать названия 12 признаков с наибольшей важностью.


Index(['BMI Category_encoded', 'systolic_pressure', 'Person ID', 'Age',
       'Heart Rate', 'Sleep Duration', 'Quality of Sleep', 'teacher',
       'Gender_encoded', 'Daily Steps', 'Sleep_eff', 'lawyer'],
      dtype='object')

В этом случае мы получили другой набор признаков.

In [115]:
X_train_selected2 = X_train[top_features2]
X_test_selected2 = X_test[top_features2]

# Обучение дерева решений на отобранных признаках
dt2 = DecisionTreeRegressor(random_state=42)
dt2.fit(X_train_selected2, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt2.predict(X_train_selected2))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt2.predict(X_test_selected2))}')

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


Результаты лучше, но 1 на тренировочных данных.
Высокое значение  R²  на тренировочных данных (1.0) в сочетании с более низким значением на тестовых данных (0.8209) может указывать на то, что модель переобучена. Это значит, что она хорошо подстраивается под конкретные данные, но может не так хорошо работать с новыми данными.

In [116]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

# • SelectKBest: Это класс из библиотеки sklearn, который используется для выбора лучших признаков на основе заданной функции оценки. Он выбирает k признаков с наибольшими оценками.
# • f_classif: Это функция оценки, которая используется для оценки значимости признаков в задачах классификации. Она вычисляет F-статистику для каждой переменной и целевой переменной, что позволяет определить, насколько хорошо каждый признак помогает предсказать целевую переменную.
# • pandas: Библиотека для работы с данными в формате таблиц, которая будет использоваться для создания DataFrame.


# Предположим, что X - ваши признаки, а y - целевая переменная.
# Создаем SelectKBest модель с выбранной функцией оценки
select_k_best = SelectKBest(score_func=f_classif, k=12)  # k - количество признаков, которые нужно выбрать
# • Здесь создается объект select_k_best, который будет использовать функцию f_classif для оценки признаков и выбирать 12 наиболее значимых признаков из набора данных.

select_k_best.fit(X, y)
# • Метод fit обучает модель на данных:
#   • X: Матрица признаков (независимые переменные).
#   • y: Вектор целевой переменной (зависимая переменная).
#  В процессе обучения модель вычисляет оценки значимости для каждого признака.

# Получаем оценки признаков
scores = select_k_best.scores_
# • Здесь извлекаются оценки значимости для всех признаков, которые были рассчитаны в процессе обучения.
# Эти оценки показывают, насколько каждый признак важен для предсказания целевой переменной.

# Создаем DataFrame для выбранных признаков и их оценок
selected_features3 = pd.DataFrame({
    'Feature': X.columns,
    'Score': scores
})
# • Создается новый DataFrame selected_features3, который содержит два столбца:
# • Feature: Названия признаков из исходного DataFrame X.
# • Score: Оценки значимости для каждого признака, полученные на предыдущем шаге.


# Сортируем по убыванию оценок
print(selected_features3.sort_values(by='Score', ascending=False))


                    Feature       Score
9      BMI Category_encoded  308.683826
21       diastolic_pressure  268.097906
15                    nurse  253.806586
20        systolic_pressure  214.585414
16              salesperson   64.394013
1                       Age   58.408963
0                 Person ID   54.653439
4   Physical Activity Level   44.150579
23                lifestyle   42.785715
19                  teacher   35.694103
6                Heart Rate   32.948639
2            Sleep Duration   31.913468
8            Gender_encoded   31.510655
22                Sleep_eff   30.914861
3          Quality of Sleep   27.600604
7               Daily Steps   24.776923
11                   doctor   19.796018
12                 engineer   18.286549
13                   lawyer   11.077560
5              Stress Level    6.603565
10               accountant    6.243088
17                scientist    1.263244
18                 software    0.531321
14                  manager    0.352653


In [117]:
top_features3 = selected_features3.sort_values(by='Score', ascending=False)[:12]
# • selected_features3.sort_values(by='Score', ascending=False): Здесь DataFrame selected_features3, который содержит названия признаков и их оценки значимости, сортируется по убыванию оценок (Score). Это позволяет расположить наиболее значимые признаки вверху списка.
# • [:12]: Этот срез выбирает верхние 12 признаков с наивысшими оценками значимости. Таким образом, top_features3 будет содержать 12 наиболее значимых признаков для модели.

X_train_selected3 = X_train[top_features3.Feature]
X_test_selected3 = X_test[top_features3.Feature]
# • X_train_selected3: Здесь создается новый DataFrame, содержащий только 12 наиболее значимых признаков (из переменной top_features3) из тренировочного набора данных X_train.
# • X_test_selected3: Аналогично, создается новый DataFrame, содержащий только те же 12 наиболее значимых признаки из тестового набора данных X_test.

# Обучение дерева решений на отобранных признаках
dt3 = DecisionTreeRegressor(random_state=42)
dt3.fit(X_train_selected3, y_train)
print(f'R^2 на тренировочных данных {r2_score(y_train, dt3.predict(X_train_selected3))}')
print(f'R^2 на тестовых данных {r2_score(y_test, dt3.predict(X_test_selected3))}')

R^2 на тренировочных данных 1.0
R^2 на тестовых данных 0.8208786231884058


▎Причины неизменности результатов

1. Переобучение:

   • Значение  R²  на тренировочных данных равное 1.0 указывает на то, что модель идеально подстраивается под тренировочные данные. Это часто происходит, когда модель слишком сложна (например, дерево решений без ограничений на глубину), и она запоминает все особенности тренировочных данных.

   • Если вы используете одно и то же дерево решений, просто изменяя набор признаков, это может не привести к значительным изменениям в производительности, если модель по-прежнему способна идеально предсказывать тренировочные данные.

2. Схожесть отобранных признаков:

   • Если 12 признаков, выбранных с помощью SelectKBest, имеют сильные корреляции с целевой переменной, они могут обеспечивать такую же предсказательную силу, как и все остальные признаки, которые вы использовали ранее. Это может привести к тому, что модель все равно будет хорошо работать на тренировочных данных.

3. Модель и данные:

   • Если структура данных не меняется, и модель остается той же (например, вы используете одно и то же дерево решений), то результаты могут оставаться стабильными. Это особенно актуально, если данные не содержат новых, непредсказуемых паттернов, которые могли бы повлиять на производительность модели.

4. Выбор признаков:

   • Возможно, выбранные 12 признаков по-прежнему достаточно информативны для модели, чтобы она могла хорошо предсказывать результаты. Если эти признаки представляют основную информацию, необходимую для предсказания, то результаты могут не измениться значительно.

В целом, с уменьшением признаков качество модели снижается. Но во всяком случае мы не получаем переобученную модель при RFE.

4. Подбор гиперпараметров:  
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.  
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

In [124]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV: Это класс из библиотеки sklearn, который используется для поиска оптимальных гиперпараметров для модели с помощью перекрестной проверки (cross-validation).
# Он перебирает все возможные комбинации заданных гиперпараметров и оценивает производительность модели для каждой комбинации.

# Функция для оценки модели с помощью перекрестной проверки
def evaluate_model(model, X_train, y_train):
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
    return scores.mean()
# • evaluate_model: Эта функция принимает модель и данные, затем использует cross_val_score для выполнения перекрестной проверки с 5 фолдами (cv=5)
# и вычисляет среднее значение коэффициента детерминации  R²  для оценки производительности модели.
# • cross_val_score: Эта функция разбивает данные на 5 частей, обучает модель на 4 частях и тестирует на 1 части,
# повторяя этот процесс 5 раз. Затем она возвращает массив оценок, из которого берется среднее значение.

# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}
# • param_grid: Это словарь, который содержит гиперпараметры, которые вы хотите оптимизировать для модели дерева решений:
# • max_depth: Максимальная глубина дерева. Ограничение глубины может помочь предотвратить переобучение.
# • min_samples_split: Минимальное количество образцов, необходимых для разделения узла. Это также может помочь предотвратить переобучение.
# • min_samples_leaf: Минимальное количество образцов, которые должны находиться в листовом узле. Это также может помочь предотвратить переобучение.



grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
# • GridSearchCV(...): Здесь создается объект grid_search, который будет использоваться для поиска лучших гиперпараметров.
#   • estimator=tree: Модель, для которой будет производиться поиск гиперпараметров.
#   • param_grid=param_grid: Словарь с гиперпараметрами для перебора.
#   • scoring='r2': Оценка, по которой будет производиться сравнение моделей (в данном случае  R² ).
#   • cv=5: Количество фолдов для перекрестной проверки.
#   • n_jobs=-1: Использовать все доступные процессоры для ускорения вычислений.
# • grid_search.fit(X_train, y_train): Обучает GridSearchCV на тренировочных данных, выполняя перебор всех комбинаций гиперпараметров и оценивая каждую из них.



print(f"Лучшие гиперпараметры для модели из коробки tree (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка для модели из коробки (Grid Search): {grid_search.best_score_:.2f}")

# Создание модели с лучшими гиперпараметрами
best_model = grid_search.best_estimator_

# Оценка модели с помощью evaluate_model
evaluation_score = evaluate_model(best_model, X_train, y_train)
print(f"Оценка лучшей модели с помощью evaluate_model: {evaluation_score:.2f}")

Лучшие гиперпараметры для модели из коробки tree (Grid Search): {'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}
Лучшая оценка для модели из коробки (Grid Search): 0.68
Оценка лучшей модели с помощью evaluate_model: 0.68


In [125]:
# Функция для оценки модели с помощью перекрестной проверки
def evaluate_model(model, X_train, y_train):
    scores = cross_val_score(model, X_train, y_train, scoring='r2', cv=5)
    return scores.mean()

# Поиск гиперпараметров с помощью сетки параметров
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 4, 6]
}

grid_search = GridSearchCV(estimator=dt1, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

print(f"Лучшие гиперпараметры для dt1 (Grid Search): {grid_search.best_params_}")
print(f"Лучшая оценка для dt1 (Grid Search): {grid_search.best_score_:.2f}")

# Создание модели с лучшими гиперпараметрами
best_model = grid_search.best_estimator_

# Оценка модели с помощью evaluate_model
evaluation_score = evaluate_model(best_model, X_train, y_train)
print(f"Оценка лучшей модели с помощью evaluate_model: {evaluation_score:.2f}")

Лучшие гиперпараметры для dt1 (Grid Search): {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Лучшая оценка для dt1 (Grid Search): 0.61
Оценка лучшей модели с помощью evaluate_model: 0.66


При кросс-валидации подтвердилось, что наша модель из коробки все же несколько переобучена, так как мы получили существенно более низкое значение 0.68.  
Модель с меньшим числом признаков кажется даже более эффективной.  
При этом для модели с меньшим числом признаков Grid Search выбрал большую глубину дерева и большее количество данных в листе, за счет чего, вероятно, и увеличил ее способность к обобщению, чтобы получить наилучший результат.